# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# useful library
import pandas as pd
import numpy as np
import random
from math import sqrt
import matplotlib.pyplot as plt

# torch stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 00 Game Info

In [ ]:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Env

In [ ]:
import os

# 파일 복사 명령어 실행
os.system('cp "/content/drive/My Drive/Colab Notebooks/공통 environment+state.ipynb" "/content/"')

0

In [ ]:
import nbformat

notebook_path = "/content/공통 environment+state.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# 각 코드 셀 출력 및 실행
for cell in notebook_content.cells:
    if cell.cell_type == "code":
        print(f"실행 중인 코드:\n{cell.source}\n{'='*40}")
        exec(cell.source)

실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 / 보드 칸
        #현재 상태에서 행동하는 행동을 수행하여 다음 상태를 계산
        next_state = present_state.next(action_idx)
        is_done, is_lose = ne

## BestModelAnalysis 모델 간 성능 비교하는 클래스
각 모델끼리 대국 붙여서 승률 비교!
한 두 번 하는 걸로는 두 모델 사이의 승률을 보기에 충분하진 않으니깐, 좀 여러 번 반복해서!

## 자료 시각화
1. 최종적으로 여러 모델 - 알파제로들(하이퍼 파라미터 설정에 따라 승률이 다를거라서 여러 개 ) 간의 승률을 비교해서 뭐가 제일 좋은 모델이었는지 자료를 시각화(or 도식화)해서 보고서에 넣기 -> 시각화 방법론 고민 ( + 여러 샘플 만들어보기 )

2. 승률 엑셀로 정리하기(판다스 사용해서 자동화할 수 있을거야) 이거 두 개야!!


## 가이드라인
1. 지는 것 / 무승부 / 이기는 것 -> 지표 3개 만들기!
2. (*이슈) 자기가 이길 생각만 하고 남이 이기는 걸 막지 못할 수가 있음 -> 한 게임 내에서 어떻게 행동하는지 지표 1개 만들기 -> 이상하다면 파라미터를 바꿔보자!
3. 1000에포크를 돌릴 때 100에포크마다 어떻게 행동하는지 지표 만들기! valid 코드도 삽입하기

-------------

In [ ]:
# 에포크 100 돌리고, 20마다 출력!

## 5개 에이전트 파일 업로드

In [ ]:
#현재 디렉터리 파일 목록 확인
!ls

 1월_27일_alphabeta.py	 1월_27일_minmax.py  '공통 environment+state.ipynb'   __pycache__
 1월_27일_alphazero.py	 alphabeta_agent.py   mcs_agent.py		      sample_data
 1월_27일_mcs.py	 alphazero_agent.py   mcts_agent.py		      train_losses.csv
 1월_27일_mcts.py	 drive		      minmax_agent.py


In [ ]:
#Google Drive로 파일 이동 + 파일 이름 변경하여 처리
!cp "1월_27일_alphazero.py" /content/alphazero_agent.py
!cp "1월_27일_minmax.py" /content/minmax_agent.py
!cp "1월_27일_mcts.py" /content/mcts_agent.py
!cp "1월_27일_mcs.py" /content/mcs_agent.py
!cp "1월_27일_alphabeta.py" /content/alphabeta_agent.py

In [ ]:
import os
import time

### 대국 실행 및 결과 저장

In [ ]:
class BestModelAnalysis:
    def __init__(self, agents, results_path='./results/validation_results.txt', ep_game_count=10):
        self.agents = agents #AlphaZero vs 에이전트 딕셔너리
        self.results_path = results_path #결과 저장 경로
        self.ep_game_count = ep_game_count #각 평가당 실행할 게임 수


    # 환경에서 정의된 보상 시스템을 그대로 사용
    def _first_player_point(self, ended_state): #ended_state: 게임이 종료된 상태
        return ended_state.get_reward(ended_state)


    #두 에이전트 간 대국 실행 및 결과 반환 (승/패/무 비율, 경기 시간, 평균 보상)
    def _play(self, next_actions):
        state = State()  # 초기 상태 생성
        # MCTS를 사용하는 경우 루트 노드 생성
        root = Node_m(state) if hasattr(next_actions[0], 'get_action_probs') else None

        start_time = time.time()  # 경기 시작 시간 기록
        moves = 0  # 턴 수 계산
        rewards = []  # 보상 기록

        while not state.check_done()[0]:
            is_first_player = state.check_first_player()

            if is_first_player:
                next_action = next_actions[0](root if isinstance(root, Node_m) else state)
            else:
                next_action = next_actions[1](root if isinstance(root, Node_m) else state)

            state = state.next(next_action)
            reward = state.get_reward(state)
            rewards.append(reward)

            # 보상 누적 과정 디버깅 추가
            print(f"[DEBUG] Move {moves+1}: Action {next_action}, Reward: {reward}")
            moves += 1  # 턴 수 증가

            # 루트 노드 업데이트 (MCTS를 사용하는 경우)
            if root and isinstance(root, Node_m):  # Node_m 타입 체크
               if next_action in root.children:
                   print(f"[DEBUG] Root updated to existing child: {next_action}")
                   root = root.children[next_action]  # 루트 노드를 자식 노드로 갱신
               elif not root.children and root.visit_count < 10:  # 탐색이 충분하지 않은 경우에만 초기화
                   print(f"[DEBUG] Root reset for new state.")
                   root = Node_m(state)  # 새 루트 노드 생성

        end_time = time.time()  # 경기 종료 시간 기록
        game_duration = end_time - start_time  # 총 경기 시간

        final_reward = state.get_reward(state) # final_reward 변수에 최종 보상 값 할당

        # 후공일 경우 보상 반전 적용
        adjusted_final_reward = final_reward if state.check_first_player() else -final_reward
        print(f"[DEBUG] Final reward before return: {final_reward}, Adjusted: {adjusted_final_reward}, is_first_player={state.check_first_player()}")

        return {
            "reward": adjusted_final_reward ,
            "duration": game_duration,
            "average_reward": sum(rewards) / len(rewards) if rewards else 0
        }


    def _evaluate_algorithm_of(self, label, next_actions):  # AlphaZero vs 다른 에이전트 대국
        total_rewards = []
        total_duration = 0
        win_count, lose_count, draw_count = 0, 0, 0

        for i in range(self.ep_game_count):
            print(f'\\nEvaluate {label} {i + 1}/{self.ep_game_count}')
            is_first_player = (i % 2 == 0)

            result = self._play(next_actions if is_first_player else list(reversed(next_actions)))
            reward = result["reward"]

            print(f"[DEBUG] Adjusted reward after processing: {reward}, is_first_player={is_first_player}")

            total_rewards.append(reward)
            total_duration += result["duration"]

            if reward == 1:
                win_count += 1
                print(f"[DEBUG] AlphaZero WIN (is_first_player={is_first_player}, reward={reward})")
            elif reward == -1:
                lose_count += 1
                print(f"[DEBUG] AlphaZero LOSE (is_first_player={is_first_player}, reward={reward})")
            else:
                draw_count += 1
                print(f"[DEBUG] DRAW (is_first_player={is_first_player}, reward={reward})")

        avg_reward = sum(total_rewards) / len(total_rewards)
        avg_duration = total_duration / self.ep_game_count

        print(f'{label}: Avg Reward: {avg_reward:.3f} | Duration: {avg_duration:.2f}s')
        print(f'Wins: {win_count}, Draws: {draw_count}, Losses: {lose_count}')

        return {
            "average_reward": avg_reward,
            "average_duration": avg_duration,
            "win_rate": win_count / self.ep_game_count,
            "draw_rate": draw_count / self.ep_game_count,
            "lose_rate": lose_count / self.ep_game_count
        }


    def run_validation(self, epoch): # AlphaZero와 다른 에이전트 간의 대결 평가 및 결과 저장
        print(f'\nEpoch {epoch} - 모델 검증 시작') #epoch: 현재 평가의 에포크 번호
        results = {} #대결 결과(평균 점수) 저장할 딕셔너리 초기화

        for name, agent in self.agents.items():
        #{'AlphaZero': AlphaZeroAgent, 'MinMax': MinMaxAgent}
            if name == 'AlphaZero':  # AlphaZero는 선 수로 고정되어 다른 에이전트와 비교
                continue

            results[f'VS_{name}'] = self._evaluate_algorithm_of( #메서드 호출하여 대결 수행
                f'VS_{name}', (self.agents['AlphaZero'].get_action, agent.get_action)
            ) #AlphaZero의 행동 함수(get_action)와 다른 에이전트의 행동 함수를 전달

        # 결과를 파일에 저장
        with open(self.results_path, 'a') as f:
            f.write(f'Epoch {epoch}:\n') #에포크 결과 기록
            for key, value in results.items(): #각 에이전트와의 대결 결과를 저장
                f.write(f'{key}: Avg Reward: {value["average_reward"]:.3f} | '
                        f'Duration: {value["average_duration"]:.2f}s | '
                        f'Win: {value["win_rate"]:.2%}, Draw: {value["draw_rate"]:.2%}, Lose: {value["lose_rate"]:.2%}\n')
            f.write('\n')

        print("결과가 저장되었습니다.")

In [ ]:
import pandas as pd

In [ ]:
class BestModelAnalysisWithPersistence(BestModelAnalysis):
    def __init__(self, agents, results_path='./results/validation_results.txt',
                 csv_path='./results/validation_results.csv', ep_game_count=10):
        super().__init__(agents, results_path, ep_game_count)
        self.csv_path = csv_path  # CSV 파일 경로


    def save_to_csv(self, epoch, results):
        """대국 결과를 CSV 파일로 저장"""
        data = []
        for key, value in results.items():
            opponent = key.replace('VS_', '')  # VS_ 접두어 제거

            # Adjusted Reward를 기반으로 승/패/무 비율 계산
            win_rate = value["win_rate"]
            draw_rate = value["draw_rate"]
            lose_rate = value["lose_rate"]

            # 승/패/무 비율이 정상적으로 저장되는지 디버깅 로그 추가
            print(f"[DEBUG] Saving to CSV - Epoch {epoch}, Opponent: {opponent}, "
                  f"Win Rate: {win_rate:.2%}, "
                  f"Draw Rate: {draw_rate:.2%}, "
                  f"Lose Rate: {lose_rate:.2%}")


            # 새로운 데이터 구조 추가 (평균 보상, 경기 시간, 승/패/무 비율 포함)
            data.append({
                'Epoch': epoch,
                'Model': 'AlphaZero',
                'Opponent': opponent,
                'Win Rate': win_rate,
                'Draw Rate': draw_rate,
                'Lose Rate': lose_rate,
                'Avg Reward': value["average_reward"],
                'Duration (s)': value["average_duration"]
            })

        # 기존 CSV 파일이 있으면 안전하게 읽고 추가 저장
        try:
            if os.path.exists(self.csv_path):
                existing_data = pd.read_csv(self.csv_path)
                df = pd.concat([existing_data, pd.DataFrame(data)], ignore_index=True)
            else:
                df = pd.DataFrame(data)

            df.to_csv(self.csv_path, index=False)
            print(f"Epoch {epoch} 결과가 CSV 파일에 저장되었습니다: {self.csv_path}")

        except Exception as e:
            print(f" CSV 파일 저장 중 오류 발생: {e}")


    def parse_results_from_csv(self):
        """CSV 파일에서 데이터를 불러와서 기존 결과를 딕셔너리 형태로 변환"""
        if not os.path.exists(self.csv_path):
            print(" CSV 파일이 존재하지 않습니다.")
            return {}  # 파일이 없으면 빈 딕셔너리 반환

        try:
            df = pd.read_csv(self.csv_path)
            cached_results = {}

            for _, row in df.iterrows():
                epoch = int(row['Epoch'])
                if epoch not in cached_results:
                    cached_results[epoch] = {}

                cached_results[epoch][f"VS_{row['Opponent']}"] = {
                    "win_rate": float(row['Win Rate']),
                    "draw_rate": float(row['Draw Rate']),
                    "lose_rate": float(row['Lose Rate']),
                    "average_reward": float(row['Avg Reward']),
                    "average_duration": float(row['Duration (s)'])
                }

            return cached_results

        except Exception as e:
            print(f" CSV 파일을 읽는 중 오류 발생: {e}")
            return {}


    def validate_model(self, epoch):
        """항상 새로운 대국을 실행하고 결과를 저장"""
        print(f'\nEpoch {epoch} - 모델 검증 시작')

        results = {}  # 새로운 결과 저장
        for name, agent in self.agents.items():
            if name == 'AlphaZero':  # AlphaZero는 항상 선 수로 고정됨
                continue
            results[f'VS_{name}'] = self._evaluate_algorithm_of(
                f'VS_{name}', (self.agents['AlphaZero'].get_action, agent.get_action)
            )

        # CSV에 저장
        self.save_to_csv(epoch, results)

        # 저장된 데이터 개수 확인
        df = pd.read_csv(self.csv_path) if os.path.exists(self.csv_path) else None
        num_rows = len(df) if df is not None else 0
        print(f"Epoch {epoch} 결과 저장 완료. 현재 CSV 데이터 개수: {num_rows} 행\n")


## Test Code

In [ ]:
import sys

In [ ]:
# 강제로 캐시 제거
if "minmax_agent" in sys.modules:
    del sys.modules["minmax_agent"]
if "mcts_agent" in sys.modules:
    del sys.modules["mcts_agent"]
if "mcs_agent" in sys.modules:
    del sys.modules["mcs_agent"]
if "alphabeta_agent" in sys.modules:
    del sys.modules["alphabeta_agent"]

In [ ]:
# 환경 객체 생성
env = Environment()

In [ ]:
# 에이전트 초기화
from alphazero_agent import AlphaZeroAgent, Network, predict
from minmax_agent import MinMaxAgent, TicTacToeEvaluator
from mcts_agent import MCTSAgent, Node_m
from mcs_agent import MCSAgent
from alphabeta_agent import AlphaBetaAgent

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5000, 1, 1])) that is different to the input size (torch.Size([5000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[DEBUG] Epoch 0, Loss: 0.0688
[DEBUG] Epoch 10, Loss: 0.0607
[DEBUG] Epoch 20, Loss: 0.0607
[DEBUG] Epoch 30, Loss: 0.0607
[DEBUG] Epoch 40, Loss: 0.0604
[DEBUG] Epoch 50, Loss: 0.0604
[DEBUG] Epoch 60, Loss: 0.0604
[DEBUG] Epoch 70, Loss: 0.0604
[DEBUG] Epoch 80, Loss: 0.0604
[DEBUG] Epoch 90, Loss: 0.0604
[DEBUG] TicTacToeEvaluator training completed!
초기 상태 보드:
Minmax가 선택한 최적의 행동: 4
테스트 상태 보드:
테스트 상태에서 Minmax 선택 행동: 0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.

## AlphaZero VS MCTS (추가)

In [ ]:
if __name__ == '__main__':
    import os
    import torch

    # 필요한 상수 정의 (필요한 경우 기존 코드에서 가져옴)
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N_RESIDUAL_BLOCK = 16
    N_KERNEL = 128
    STATE_SIZE = (3, 3)
    N_ACTIONS = 9

    # 결과 디렉토리 생성
    results_dir = os.path.dirname('./results/validation_results.txt')
    os.makedirs(results_dir, exist_ok=True)

    # 디렉토리 내 모든 체크포인트 파일 검색
    model_dir = '/content/drive/My Drive/3-2/강화'
    checkpoint_files = [f for f in os.listdir(model_dir) if f.startswith('model_env_checkpoint_epoch_') and f.endswith('.pth')]

    # 에포크 번호로 정렬하여 최신 파일 선택
    checkpoint_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest_checkpoint = checkpoint_files[-1]

    # 최신 체크포인트 파일 경로
    model_path = os.path.join(model_dir, latest_checkpoint)
    print(f"가장 최신 체크포인트 파일: {model_path}")

    # AlphaZero 모델 초기화
    alphazero_model = Network(N_RESIDUAL_BLOCK, N_KERNEL, STATE_SIZE, N_ACTIONS)

    # 모델을 DataParallel로 래핑
    alphazero_model = torch.nn.DataParallel(alphazero_model).to(DEVICE)

    # 저장된 모델 가중치 로드
    #현재 초기화된 모델에 맞게 동기화하려면, 현재 모델의 구조와 키를 확인해야함
    params = torch.load(model_path, map_location=torch.device('cpu'))

    # 현재 모델의 키 가져오기
    model_keys = alphazero_model.module.state_dict().keys()

    # 저장된 가중치의 키와 현재 모델 키를 동기화
    if any(k.startswith("module.") for k in model_keys) and not any(k.startswith("module.") for k in params.keys()):
        # 현재 모델 키에 "module."이 있으나 저장된 키에 없을 경우
        params = {f"module.{k}": v for k, v in params.items()}
    elif not any(k.startswith("module.") for k in model_keys) and any(k.startswith("module.") for k in params.keys()):
        # 현재 모델 키에 "module."이 없으나 저장된 키에 있을 경우
        params = {k.replace("module.", ""): v for k, v in params.items()}

    # 가중치 로드
    alphazero_model.load_state_dict(params, strict=False)  # 키 불일치 허용

    # AlphaZeroAgent 초기화 및 평가 설정
    alphazero_agent = AlphaZeroAgent(
        model_path=model_path,         # 학습된 모델 불러오기
        eval_game_count=30,            # 평가 게임 수
        pv_evaluate_count=100,         # MCTS 탐색 횟수 (대국 성능에 영향)
        env=env,                       # 게임 환경 정보
        sp_game_count=0,               # Self-play 게임 없음
        learning_rate=0.01,            # 학습률 기본값 유지
        batch_size=64,                 # 배치 크기 기본값 유지
        epochs=30,                     # 학습 없음
        learn_epoch=0                  # 학습 없음
    )

    alphazero_agent.selfplay(execute_self_play=False)  # self-play 비활성화

    # TicTacToeEvaluator 초기화
    evaluator = TicTacToeEvaluator(input_shape=(3, 3), hidden_units=128).to(DEVICE)

    # 에이전트 딕셔너리 생성
    agents = {
        'AlphaZero': alphazero_agent,
        'MCTS': MCTSAgent(policy_model=None)
    }

    # BestModelAnalysisWithPersistence 사용 (CSV 저장)
    analysis = BestModelAnalysisWithPersistence(
        agents=agents,
        results_path='./results/validation_results.txt',
        csv_path='./results/validation_results.csv',
        ep_game_count=10
    )

    # 에포크별 평가 실행 (기존 데이터 체크 후 실행)
    for epoch in range(0, 51, 10):
        analysis.validate_model(epoch)

    print('모든 검증 완료')
    print("대국 결과가 CSV에 저장되었습니다.")

    # **최종 결과 출력 (CSV에서 불러와서 출력)**
    print("\n=== 최종 대국 결과 ===")
    cached_results = analysis.parse_results_from_csv()
    for epoch, results in cached_results.items():
        print(f'\nEpoch {epoch} 결과:')
        for opponent, values in results.items():
            print(f'VS {opponent} -> '
                  f'Win Rate: {values["win_rate"]:.2%}, '
                  f'Draw Rate: {values["draw_rate"]:.2%}, '
                  f'Lose Rate: {values["lose_rate"]:.2%}, '
                  f'Avg Reward: {values["average_reward"]:.3f}, '
                  f'Avg Duration: {values["average_duration"]:.2f}s')

    print("\n모든 검증 과정이 완료되었습니다!")

가장 최신 체크포인트 파일: /content/drive/My Drive/3-2/강화/model_env_checkpoint_epoch_17.pth
Self-play 실행이 비활성화되었습니다.

Epoch 0 - 모델 검증 시작
\nEvaluate VS_MCTS 1/10


<ipython-input-179-ad0c0404f63e>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(model_path, map_location=torch.device('cpu'))


[DEBUG] Move 1: Action 3, Reward: 0
[DEBUG] Move 2: Action 2, Reward: 0
[DEBUG] Move 3: Action 4, Reward: 0
[DEBUG] Move 4: Action 0, Reward: 0
[DEBUG] Move 5: Action 5, Reward: -1
[DEBUG] Final reward before return: -1, Adjusted: 1, is_first_player=False
[DEBUG] Adjusted reward after processing: 1, is_first_player=True
[DEBUG] AlphaZero WIN (is_first_player=True, reward=1)
\nEvaluate VS_MCTS 2/10
[DEBUG] Move 1: Action 8, Reward: 0
[DEBUG] Move 2: Action 1, Reward: 0
[DEBUG] Move 3: Action 6, Reward: 0
[DEBUG] Move 4: Action 3, Reward: 0
[DEBUG] Move 5: Action 3, Reward: 0
[DEBUG] Move 6: Action 5, Reward: 0
[DEBUG] Move 7: Action 3, Reward: 0
[DEBUG] Move 8: Action 4, Reward: -1
[DEBUG] Final reward before return: -1, Adjusted: -1, is_first_player=True
[DEBUG] Adjusted reward after processing: -1, is_first_player=False
[DEBUG] AlphaZero LOSE (is_first_player=False, reward=-1)
\nEvaluate VS_MCTS 3/10
[DEBUG] Move 1: Action 4, Reward: 0
[DEBUG] Move 2: Action 4, Reward: 0
[DEBUG] Move 

## AlphaZero vs 나머지 3개 에이전트

In [ ]:
if __name__ == '__main__':
    import os
    import torch

    # 필요한 상수 정의 (필요한 경우 기존 코드에서 가져옴)
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    N_RESIDUAL_BLOCK = 16
    N_KERNEL = 128
    STATE_SIZE = (3, 3)
    N_ACTIONS = 9

    # 결과 디렉토리 생성
    results_dir = os.path.dirname('./results/validation_results.txt')
    os.makedirs(results_dir, exist_ok=True)

    # 디렉토리 내 모든 체크포인트 파일 검색
    model_dir = '/content/drive/My Drive/3-2/강화'
    checkpoint_files = [f for f in os.listdir(model_dir) if f.startswith('model_env_checkpoint_epoch_') and f.endswith('.pth')]

    # 에포크 번호로 정렬하여 최신 파일 선택
    checkpoint_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest_checkpoint = checkpoint_files[-1]

    # 최신 체크포인트 파일 경로
    model_path = os.path.join(model_dir, latest_checkpoint)
    print(f"가장 최신 체크포인트 파일: {model_path}")

    # AlphaZero 모델 초기화
    alphazero_model = Network(N_RESIDUAL_BLOCK, N_KERNEL, STATE_SIZE, N_ACTIONS)

    # 모델을 DataParallel로 래핑
    alphazero_model = torch.nn.DataParallel(alphazero_model).to(DEVICE)

    # 저장된 모델 가중치 로드
    #현재 초기화된 모델에 맞게 동기화하려면, 현재 모델의 구조와 키를 확인해야함
    params = torch.load(model_path, map_location=torch.device('cpu'))

    # 현재 모델의 키 가져오기
    model_keys = alphazero_model.module.state_dict().keys()

    # 저장된 가중치의 키와 현재 모델 키를 동기화
    if any(k.startswith("module.") for k in model_keys) and not any(k.startswith("module.") for k in params.keys()):
        # 현재 모델 키에 "module."이 있으나 저장된 키에 없을 경우
        params = {f"module.{k}": v for k, v in params.items()}
    elif not any(k.startswith("module.") for k in model_keys) and any(k.startswith("module.") for k in params.keys()):
        # 현재 모델 키에 "module."이 없으나 저장된 키에 있을 경우
        params = {k.replace("module.", ""): v for k, v in params.items()}

    # 가중치 로드
    alphazero_model.load_state_dict(params, strict=False)  # 키 불일치 허용

    # AlphaZeroAgent 초기화 및 평가 설정
    alphazero_agent = AlphaZeroAgent(
        model_path=model_path,         # 학습된 모델 불러오기
        eval_game_count=30,            # 평가 게임 수
        pv_evaluate_count=100,         # MCTS 탐색 횟수 (대국 성능에 영향)
        env=env,                       # 게임 환경 정보
        sp_game_count=0,               # Self-play 게임 없음
        learning_rate=0.01,            # 학습률 기본값 유지
        batch_size=64,                 # 배치 크기 기본값 유지
        epochs=30,                     # 학습 없음
        learn_epoch=0                  # 학습 없음
    )

    alphazero_agent.selfplay(execute_self_play=False)  # self-play 비활성화

    # TicTacToeEvaluator 초기화
    evaluator = TicTacToeEvaluator(input_shape=(3, 3), hidden_units=128).to(DEVICE)

    # 에이전트 딕셔너리 생성
    agents = {
        'AlphaZero': alphazero_agent,
        'MinMax': MinMaxAgent(evaluator=evaluator, environment=env),
        'MCS': MCSAgent(),
        'AlphaBeta': AlphaBetaAgent()
    }

    # BestModelAnalysisWithPersistence 사용 (CSV 저장)
    analysis = BestModelAnalysisWithPersistence(
        agents=agents,
        results_path='./results/validation_results.txt',
        csv_path='./results/validation_results.csv',
        ep_game_count=10
    )

    # 에포크별 평가 실행 (기존 데이터 체크 후 실행)
    for epoch in range(0, 51, 10):
        analysis.validate_model(epoch)

    print('모든 검증 완료')
    print("대국 결과가 CSV에 저장되었습니다.")

    # **최종 결과 출력 (CSV에서 불러와서 출력)**
    print("\n=== 최종 대국 결과 ===")
    cached_results = analysis.parse_results_from_csv()
    for epoch, results in cached_results.items():
        print(f'\nEpoch {epoch} 결과:')
        for opponent, values in results.items():
            print(f'VS {opponent} -> '
                  f'Win Rate: {values["win_rate"]:.2%}, '
                  f'Draw Rate: {values["draw_rate"]:.2%}, '
                  f'Lose Rate: {values["lose_rate"]:.2%}, '
                  f'Avg Reward: {values["average_reward"]:.3f}, '
                  f'Avg Duration: {values["average_duration"]:.2f}s')

    print("\n모든 검증 과정이 완료되었습니다!")

가장 최신 체크포인트 파일: /content/drive/My Drive/3-2/강화/model_env_checkpoint_epoch_17.pth
Self-play 실행이 비활성화되었습니다.

Epoch 0 - 모델 검증 시작
\nEvaluate VS_MinMax 1/10


<ipython-input-182-903dc5b19e58>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(model_path, map_location=torch.device('cpu'))


[DEBUG] Move 1: Action 8, Reward: 0
[DEBUG] Move 2: Action 0, Reward: 0
[DEBUG] Move 3: Action 2, Reward: 0
[DEBUG] Move 4: Action 6, Reward: 0
[DEBUG] Move 5: Action 5, Reward: -1
[DEBUG] Final reward before return: -1, Adjusted: 1, is_first_player=False
[DEBUG] Adjusted reward after processing: 1, is_first_player=True
[DEBUG] AlphaZero WIN (is_first_player=True, reward=1)
\nEvaluate VS_MinMax 2/10
[DEBUG] Move 1: Action 0, Reward: 0
[DEBUG] Move 2: Action 1, Reward: 0
[DEBUG] Move 3: Action 8, Reward: 0
[DEBUG] Move 4: Action 4, Reward: 0
[DEBUG] Move 5: Action 2, Reward: 0
[DEBUG] Move 6: Action 5, Reward: 0
[DEBUG] Move 7: Action 3, Reward: 0
[DEBUG] Move 8: Action 7, Reward: -1
[DEBUG] Final reward before return: -1, Adjusted: -1, is_first_player=True
[DEBUG] Adjusted reward after processing: -1, is_first_player=False
[DEBUG] AlphaZero LOSE (is_first_player=False, reward=-1)
\nEvaluate VS_MinMax 3/10
[DEBUG] Move 1: Action 0, Reward: 0
[DEBUG] Move 2: Action 2, Reward: 0
[DEBUG] M